In [47]:
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt
import keras

In [9]:
IMAGE_SIZE=256
BATCH_SIZE=32

In [10]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
  "PlantVillage",
  shuffle=True,
  image_size=(IMAGE_SIZE,IMAGE_SIZE),
  batch_size=BATCH_SIZE
)

Found 2152 files belonging to 3 classes.


In [11]:
class_names=dataset.class_names

In [12]:
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

Train-test split

In [13]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [14]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [15]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [52]:
resize_and_rescale = tf.keras.Sequential([
  keras.layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  keras.layers.Rescaling(1./255),
])

In [53]:
data_augmentation = tf.keras.Sequential([
  keras.layers.RandomFlip("horizontal_and_vertical"),
  keras.layers.RandomRotation(0.2),
])

In [55]:
model=tf.keras.Sequential([
  resize_and_rescale,
    keras.layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])



c:\Users\rayqu\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [56]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [57]:
model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=50,
)

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 215s 4s/step - accuracy: 0.4818 - loss: 0.9451 - val_accuracy: 0.6458 - val_loss: 0.8326
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.6341 - loss: 0.7591 - val_accuracy: 0.6667 - val_loss: 0.6667
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 753s 14s/step - accuracy: 0.7503 - loss: 0.5575 - val_accuracy: 0.8854 - val_loss: 0.3705
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.8369 - loss: 0.4253 - val_accuracy: 0.8802 - val_loss: 0.2974
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - accuracy: 0.8800 - loss: 0.3067 - val_accuracy: 0.9375 - val_loss: 0.1532
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 231s 4s/step - accuracy: 0.9179 - loss: 0.1945 - val_accuracy: 0.9792 - val_loss: 0.0731
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - accuracy: 0.9471 - loss: 0.1289 - val_accuracy: 0.9323 - val_loss: 0.1218
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.9687 - loss: 0.0784 - val_accuracy: 0.9583 - val_l

In [58]:
model.evaluate(test_ds)

8/8 ━━━━━━━━━━━━━━━━━━━━ 28s 641ms/step - accuracy: 0.9901 - loss: 0.0698


[0.028437787666916847, 0.99609375]

In [62]:
import pickle

with open('potato_disease_classifier.pkl','wb') as file:
  pickle.dump(model,file)